In [39]:
import os
import sys
import urllib.request
client_id = "ljNG6p8N6mET1oVTCu4F"
client_secret = "RFboX9njKl"
keyword = input("여행할 지역과 관련된 키워드를 입력하세요 : ")
encText = urllib.parse.quote(keyword + ' 여행 후기')
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id","ljNG6p8N6mET1oVTCu4F")
request.add_header("X-Naver-Client-Secret","RFboX9njKl")
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

여행할 지역과 관련된 키워드를 입력하세요 : 부산
{
"lastBuildDate": "Wed, 09 Feb 2022 03:28:19 +0900",
"total": 82814,
"start": 1,
"display": 10,
"items": [
{
"title": "<b>부산 여행후기</b> 49편 - 해운대 맛집 : 혜성 막창 &amp; 전골",
"link": "https:\/\/blog.naver.com\/houself?Redirect=Log&logNo=222615726301",
"description": "<b>부산 여행후기</b> 49편 - 혜성막창 00 :: 워낙 유명한 곳이라서, 더이상 설명이 필요할까 싶지만, 개인적으론 약간 아쉬운 점이 있어서 ㅎㅎㅎㅎ + 필자의 모든 여행은 100% 제 돈으로만 갑니다 01 :: 위치 해운대에... ",
"bloggername": "계장쟁이 @ 면봉",
"bloggerlink": "https://blog.naver.com/houself",
"postdate": "20220107"

},
{
"title": "주차, <b>후기</b> / 그린레일웨이 산책 / <b>부산</b> 청사포 오션뷰 카페... ",
"link": "https:\/\/blog.naver.com\/jina0676?Redirect=Log&logNo=222636986065",
"description": "블루라인파크 <b>후기</b>는 이만 마무리 하고 <b>부산 여행</b>의 하이라이트, 요트 투어는 다음 포스팅에서 계속~! ▼ 다음 포스트 맛보기 사진~! #<b>부산</b>, #<b>부산여행</b>, #겨울<b>여행</b>, #겨울<b>부산여행</b>, #<b>부산</b>가볼만한곳, #<b>부산</b>맛집... ",
"bloggername": "트리안's 여행과 사진놀이",
"bloggerlink": "https://blog.naver.com/jina0676",
"postdate": "20220205"

},
{
"title": "그랜드조선 <b

In [40]:
import requests
import openpyxl

client_id = "ljNG6p8N6mET1oVTCu4F"
client_secret = "RFboX9njKl"
start, num = 1, 0

excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active
excel_sheet.column_dimensions['B'].width = 100
excel_sheet.column_dimensions['C'].width = 100
excel_sheet.append(['순서', '제목', '링크'])

for index in range(10):
    start_number = start + (index * 100)
    blog_url = 'https://openapi.naver.com/v1/search/blog.json?query='+keyword+"&display=100&start=" + str(start_number)
    header_params = {"X-Naver-Client-Id":client_id, "X-Naver-Client-Secret":client_secret}
    res = requests.get(blog_url, headers=header_params)
    if res.status_code == 200:
        data = res.json()
        for item in data['items']:
            num += 1
            excel_sheet.append([num, item['title'].replace("<b>","").replace("</b>",""), item['link']])
    else:
        print ("Error Code:", res.status_code)

excel_file.save(keyword+' 여행후기list.xlsx')
excel_file.close()

## 데이터 프레임으로 만들기

In [50]:
import pandas as pd
import requests
from urllib.parse import urlparse

url = "https://openapi.naver.com/v1/search/blog?query="+ keyword
result = requests.get(urlparse(url).geturl(),
         headers = {"X-Naver-Client-Id" : "ljNG6p8N6mET1oVTCu4F",
                                 "X-Naver-Client-Secret" : "RFboX9njKl"})

json_obj = result.json()

In [51]:
title = []
blog_url_list = []

client_id = "ljNG6p8N6mET1oVTCu4F"
client_secret = "RFboX9njKl"
start, num = 1, 0


for index in range(10):
    start_number = start + (index * 100)
    blog_url = 'https://openapi.naver.com/v1/search/blog.json?query='+keyword+"&display=100&start=" + str(start_number)
    header_params = {"X-Naver-Client-Id":client_id, "X-Naver-Client-Secret":client_secret}
    res = requests.get(blog_url, headers=header_params)
    for item in data['items']:
        title.append(item['title'].replace("<b>","").replace("</b>",""))
        blog_url_list.append(item['link'])



In [62]:
blog = {
    '제목' : title,
    '주소' : blog_url_list
}

In [63]:
pd.DataFrame(blog)
blog_df = pd.DataFrame(blog)

In [64]:

blog_title = blog_df['제목'].values.tolist()
blog_juso = blog_df['주소'].values.tolist()


### DB저장

In [70]:
import psycopg2

In [71]:
conn_str = "host=localhost dbname =postgres user=postgres password=admin port = 5432"

try : 
    conn = psycopg2.connect(conn_str)
    print("==== 접속 성공 ====")
except psycopg2.DatabaseError as db_err :
    print("접속오류 !!")
    print(db_err)


==== 접속 성공 ====


In [72]:
cur = conn.cursor()

In [77]:
for i in range(len(blog_title)) :

    try :
        cur.execute("INSERT INTO blog_result(blog_title, blog_url) VALUES (%s, %s);",
        (blog_title[i], blog_juso[i]))
        conn.commit()
        print("----- insert ok")
    except : 
        conn.rollback()
        print("---DB error")

----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- in

----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- in